In [16]:
import pandas as pd
import numpy as np
import importlib
import awswrangler as wr
from application.modeling import helper
from application.modeling.constants import MY_SESSION, S3_CLIENT, FileStage


pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

In [17]:
importlib.reload(helper)

<module 'application.modeling.helper' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\helper.py'>

In [18]:
bucket_name = "zimnat-budgeting"
tenant_name = bucket_name
project_id = 2
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12

In [24]:
file_stage = FileStage.raw
details_of_new_short_term_borrowing = wr.s3.read_parquet(
    f"s3://{bucket_name}/project_{project_id}/{file_stage.value}/details_of_new_short_term_borrowing.parquet",
    boto3_session=MY_SESSION,
)


details_of_new_short_term_borrowing

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,12,Monthly (P+I),12,reducing_balance
1,06/05/2024,MASAWARA,5613,0.06,12,Monthly (P+I),12,reducing_balance
